## Quick Intro to Large Language Models

## Setup

In [20]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_BASE = os.environ["COMPLETIONS_API_BASE"]
SYSTEM_PROMPT_TEMPLATE = "You are an AI assistant that follows instruction extremely well. Help as much as you can."
# orca-mini-v3
def get_prompt_template(instruction: str):
    prompt = f"### System:\n{SYSTEM_PROMPT_TEMPLATE}\n\n### User:\n{instruction}\n\n### Assistant:\n"
    return prompt

### Tracking Token Usage

In [29]:
%%time

from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback

llm = OpenAI(openai_api_base=OPENAI_API_BASE, n=2, best_of=2)

with get_openai_callback() as cb:
    result = llm(get_prompt_template("Tell me a joke."))
    print(result)
    print(cb)

 A man walks into a bar with his pet monkey. The bartender looks up and says, "Hey, I've never seen anything like this in my whole life." The man explains, "Yeah, he's a great drinker." The bartender raises an eyebrow and asks, "Really? What does he drink?" The man answers, "Long Island Iced Tea."
Tokens Used: 137
	Prompt Tokens: 46
	Completion Tokens: 91
Successful Requests: 1
Total Cost (USD): $0.0027400000000000002
CPU times: user 156 ms, sys: 4.83 ms, total: 161 ms
Wall time: 25.5 s


In the OpenAI API, the `n` parameter specifies the number of completions to generate for a given prompt, while the `best_of` parameter controls the number of candidate completions generated server-side⁶. The API then returns the `n` "best" completions (the ones with the highest log probability per token) out of the `best_of` candidates⁶.

In the code you provided, both `n` and `best_of` are set to 2, which means that the API will generate 2 candidate completions and return the 2 "best" ones. Is there anything else you would like to know? 😊

Source: Conversation with Bing, 8/17/2023
(1) undefined. https://platform.openai.com/docs/api-reference/introduction.
(2) OpenAI Platform. https://platform.openai.com/docs/api-reference/introduction..
(3) How is the search among 'best_of' for 'n' completions conducted in the .... https://stackoverflow.com/questions/76009968/how-is-the-search-among-best-of-for-n-completions-conducted-in-the-openai-co.
(4) What is the OpenAI API warning: To avoid an invalid_request_error, best .... https://stackoverflow.com/questions/75196860/what-is-the-openai-api-warning-to-avoid-an-invalid-request-error-best-of-was-s.
(5) undefined. https://api.openai.com/v1/models.
(6) undefined. https://api.openai.com/v1/chat/completions.

### Few-shot learning

In [5]:
from langchain import PromptTemplate
from langchain import FewShotPromptTemplate

# create our examples
examples = [
    {
        "query": "What's the weather like?",
        "answer": "It's raining cats and dogs, better bring an umbrella!"
    }, {
        "query": "How old are you?",
        "answer": "Age is just a number, but I'm timeless."
    }
]

# create an example template
example_template = """
User: {query}
AI: {answer}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are excerpts from conversations with an AI
assistant. The assistant is known for its humor and wit, providing
entertaining and amusing responses to users' questions. Here are some
examples:
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

# now create the few-shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain

# load the model
chat = ChatOpenAI(openai_api_base=OPENAI_API_BASE,  
                  temperature=0.0,
                  stop=["\n\n"], 
                  max_tokens=128)

chain = LLMChain(llm=chat, prompt=few_shot_prompt_template, verbose=True)
chain.run("What's the meaning of life?")

WARNING! stop is not default parameter.
                    stop was transferred to model_kwargs.
                    Please confirm that stop is what you intended.




> Entering new  chain...
Prompt after formatting:
The following are excerpts from conversations with an AI
assistant. The assistant is known for its humor and wit, providing
entertaining and amusing responses to users' questions. Here are some
examples:



User: What's the weather like?
AI: It's raining cats and dogs, better bring an umbrella!



User: How old are you?
AI: Age is just a number, but I'm timeless.



User: What's the meaning of life?
AI: 

> Finished chain.


' The meaning of life is to find out what makes you happy and do that as often as possible.'

### Examples with Easy Prompts: Text Summarization, Text Translation, and Question Answering


#### Creating a Question-Answering Prompt Template

In [23]:
from langchain import PromptTemplate

template = """Question: {question}

Answer: """
prompt = PromptTemplate(
    template=get_prompt_template(template),
    input_variables=['question']
)

# user question
question = "What is the capital city of France?"

In [26]:
%%time

llm_chain = LLMChain(
    prompt=prompt,
    llm=chat,
    verbose=True
)

# ask the user question about the capital of France
print(llm_chain.run(question))



> Entering new  chain...
Prompt after formatting:
### System:
You are an AI assistant that follows instruction extremely well. Help as much as you can.

### User:
Question: What is the capital city of France?

Answer: 

### Assistant:


> Finished chain.
 The capital city of France is Paris.
CPU times: user 9.94 ms, sys: 996 µs, total: 10.9 ms
Wall time: 8.35 s


#### Asking Multiple Questions

In [27]:
qa = [
    {'question': "What is the capital city of France?"},
    {'question': "What is the largest mammal on Earth?"},
    {'question': "Which gas is most abundant in Earth's atmosphere?"},
    {'question': "What color is a ripe banana?"}
]
res = llm_chain.generate(qa)
print(res)

generations=[[ChatGeneration(text=' The capital city of France is Paris.', generation_info=None, message=AIMessage(content=' The capital city of France is Paris.', additional_kwargs={}, example=False))], [ChatGeneration(text=' The largest living terrestrial mammal is the African bush elephant (Loxodonta africana).', generation_info=None, message=AIMessage(content=' The largest living terrestrial mammal is the African bush elephant (Loxodonta africana).', additional_kwargs={}, example=False))], [ChatGeneration(text=" The most abundant gas in Earth's atmosphere is nitrogen (N2), making up approximately 78.09% of the total mass. Oxygen (O2) makes up the second-most abundant gas, accounting for about 20.95%.", generation_info=None, message=AIMessage(content=" The most abundant gas in Earth's atmosphere is nitrogen (N2), making up approximately 78.09% of the total mass. Oxygen (O2) makes up the second-most abundant gas, accounting for about 20.95%.", additional_kwargs={}, example=False))], 

In [28]:
%%time

multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""
long_prompt = PromptTemplate(template=get_prompt_template(multi_template), 
            input_variables=["questions"])

llm_chain = LLMChain(
    prompt=long_prompt,
    llm=llm,
    verbose=True
)

qs_str = (
    "What is the capital city of France?\n" +
    "What is the largest mammal on Earth?\n" +
    "Which gas is most abundant in Earth's atmosphere?\n" +
	"What color is a ripe banana?\n"
)

llm_chain.run(qs_str)



> Entering new  chain...
Prompt after formatting:
### System:
You are an AI assistant that follows instruction extremely well. Help as much as you can.

### User:
Answer the following questions one at a time.

Questions:
What is the capital city of France?
What is the largest mammal on Earth?
Which gas is most abundant in Earth's atmosphere?
What color is a ripe banana?


Answers:


### Assistant:


> Finished chain.
CPU times: user 11 ms, sys: 313 µs, total: 11.3 ms
Wall time: 19.9 s


" The capital city of France is Paris.\nThe largest mammal on Earth is the blue whale.\nThe most abundant gas in Earth's atmosphere is nitrogen (N2).\nA ripe banana is yellow."

#### Text Summarization

In [30]:
summarization_template = "Summarize the following text to one sentence: {text}"
summarization_prompt = PromptTemplate(input_variables=["text"], 
                            template=get_prompt_template(summarization_template))
summarization_chain = LLMChain(llm=llm, prompt=summarization_prompt)

In [31]:
text = "LangChain provides many modules that can be used to build language model applications. Modules can be combined to create more complex applications, or be used individually for simple applications. The most basic building block of LangChain is calling an LLM on some input. Let’s walk through a simple example of how to do this. For this purpose, let’s pretend we are building a service that generates a company name based on what the company makes."
summarized_text = summarization_chain.predict(text=text)
print(summarized_text)

 LangChain offers modules for language model applications and can be combined or used individually for various use cases, with a basic example demonstrating generating a company name based on its product.


#### Text Translation

In [32]:
translation_template = "Translate the following text from {source_language} to {target_language}: {text}"
translation_prompt = PromptTemplate(input_variables=["source_language", "target_language", "text"], 
template=get_prompt_template(translation_template))
translation_chain = LLMChain(llm=llm, prompt=translation_prompt)

In [33]:
source_language = "English"
target_language = "French"
text = "Your text here"
translated_text = translation_chain.predict(
    source_language=source_language, 
    target_language=target_language, 
    text=text)
print(translated_text)

 Votre texte ici
